In [1]:
!pip install stable-baselines[mpi]==2.10.0
import gym
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import math

from simple_particle import SimpleParticle # Importing Environment
from stable_baselines.common.env_checker import check_env


/home/cmb/anaconda3/envs/microrobot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cmb/anaconda3/envs/microrobot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cmb/anaconda3/envs/microrobot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cmb/anaconda3/envs/microrobot/lib/pyt

In [ ]:

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.deepq.policies import FeedForwardPolicy
from stable_baselines import DQN, PPO2, A2C, ACKTR
from stable_baselines.bench import Monitor
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines.common.noise import AdaptiveParamNoiseSpec, NormalActionNoise
%matplotlib inline

### Instantiating the Environment

In [2]:
# Setting Parameters 
# numParticles, phi, stateBounds, dwellTime, maxSteps
numParticles = 1
phi = 0 

env = SimpleParticle(numParticles = numParticles, 
               phi = phi, 
               stateBounds = [-10.0, 10.0, -5.0, 5.0],
               dwellTime = 1.0, 
               maxSteps = 25)


/home/cmb/anaconda3/envs/microrobot/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
env.reset()

[ 3.83903001  3.59059885 -0.13968004  0.22096819]


array([ 3.83903001,  3.59059885, -0.13968004,  0.22096819])

In [4]:
env.step(0)

(array([8.83903  , 3.5905988, 5.       , 0.       ], dtype=float32),
 -9.540485,
 False,
 {})

In [5]:
# Check the environment
env_test = SimpleParticle(numParticles = numParticles, 
               phi = phi, 
               stateBounds = [-10.0, 10.0, -5.0, 5.0],
               dwellTime = 1.0, 
               maxSteps = 25)

# Check
check_env(env_test, warn=True)

[-3.00195747 -1.36896774 -0.82617816  0.97185017]


AssertionError: The reward returned by `step()` must be a float

In [6]:
env.reset()

[1.63218062 0.95481342 0.92570784 0.06759725]


array([1.63218062, 0.95481342, 0.92570784, 0.06759725])